**lab 04: Naive Bayes**

Ta  có $X_i=(x_i^1,x_i^2,...,x_i^n)$ là một sample có n features, với $c$ labels ($C_1,C_2,...C_c$). Ta muốn chọn được label cho sample $X_i$ thì dựa vào:
$$C=\mathop{argmax}_{C_j, j=\overline{1,c}}P(C_j|X_i)$$
Giả sử các thành phần của biến ngẫu nhiên X độc lập với nhau, ta được: 
$$C=\mathop{argmax}_{C_j, j=\overline{1,c}}P(C_j)\prod_{i=1}^n P(X_i|C_j)$$

Tóm lại: với mỗi label $C_j$ ta tính tích $P(C_j)\prod_{i=1}^n P(X_i|C_j)$ và so sánh chúng với nhau từ đó chọn ra được label mang giá trị lớn nhất

Một số phân phối:
1. Multinomial

2. Gaussian


In [1]:
import pandas as pd
import numpy as np

from sklearn import metrics

In [2]:
data=pd.read_csv('https://raw.githubusercontent.com/huynhthanh98/ML/master/lab-04/train.csv')
data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [3]:
print(data.shape)

(891, 12)


In [4]:
# Drop the columns "Cabin" and "Age"
data = data.drop(["Cabin","Age"],axis=1)
# Drop the NaN value
data = data.dropna()
data.shape

(889, 10)

In [5]:
# Split data by using the sklearn
from sklearn.model_selection import train_test_split
data_train, data_test = train_test_split(data, test_size=0.2, 
                                        random_state=42)

In [6]:
# Analize the value in each feature
for i in data_train.drop(labels=["Name", "Ticket","PassengerId"],axis=1).columns:
    print(i)
    print(data_train[i].unique())

Survived
[1 0]
Pclass
[1 3 2]
Sex
['female' 'male']
SibSp
[0 1 8 5 4 2 3]
Parch
[0 2 1 3 6 4 5]
Fare
[151.55    14.4542   7.925   69.55    46.9     31.275    7.8958  35.5
   0.      26.      23.25    15.5      7.7875  13.5      6.4958  26.55
   7.5208   7.2292  53.1     25.4667  31.3875   8.6625  34.0208   7.125
  52.       7.8875  14.4583  83.1583   8.05    90.       7.725   39.6
  21.075    7.7333   7.25    77.2875   9.5     12.65    10.5     55.
   7.6292   7.75     7.8542  24.15    57.9792  15.      39.6875   7.225
 227.525   17.8      9.5875 113.275    7.8292  13.      30.5      9.
  71.       4.0125   7.05    78.85     8.85    27.75    27.7208 153.4625
   8.3     19.2583  14.5    133.65    76.7292  26.2833  20.525    7.8792
  29.7     15.7417  30.6958  51.8625  81.8583  56.4958  39.       7.4958
   7.7958   7.55    12.      93.5     21.      12.35    16.1     78.2667
  35.      55.9      7.8     30.      18.      26.2875  83.475   59.4
  77.9583  28.7125  56.9292   7.65    22.358

## 1. Dùng các features: Sex, Pclass, và Embarked để xây dựng thuật toán Naive Bayes Multinomial. 

In [7]:
def get_features(data):
    """
        Purpose: get feature from data
        Input: dataframe
        Output: features
    """
    Survived=data["Survived"].values
    Pclass=data["Pclass"].values
    Sex=data["Sex"].values
    SibSp=data["SibSp"].values
    Parch=data["Parch"].values
    Embarked=data["Embarked"].values
    Fare=data["Fare"].values
    return Survived, Pclass, Sex, SibSp, Parch, Embarked,Fare

In [8]:
Survived_train, Pclass_train, Sex_train, SibSp_train, Parch_train,Embarked_train,Fare_train=get_features(data = data_train)
Survived_test, Pclass_test, Sex_test, SibSp_test, Parch_test,Embarked_test,Fare_test=get_features(data = data_test)

$P(Feature)*P(X|Feature)$

Build algorithm with Multinomial

In [9]:
#Xác xuất survived và notsurvived
# P_Survived=0.5
# P_notSurvived=0.5
# Tính xác suất từ label
data1_train=data_train[['Sex','Pclass','Embarked','Survived']]
data1_test=data_test[['Sex','Pclass','Embarked','Survived']]
P_Survived=data1_train.Survived.value_counts()[1]/len(data1_train.Survived)
P_notSurvived=data1_train.Survived.value_counts()[0]/len(data1_train.Survived)
##Survived
P_SexisMale_Survived= np.sum((Sex_train[Survived_train==1]=="male"))/np.sum((Survived_train==1))
P_SexisFemale_Survived=np.sum((Sex_train[Survived_train==1]=="female"))/np.sum((Survived_train==1))
##Tính xác xuất hậu nghiệm trên tập (P(X|Survived) trên tập test)
P_Sex_Survived= P_SexisMale_Survived*(Sex_test=="male").astype(float) +P_SexisFemale_Survived*(Sex_test=="female").astype(float)

##notSurvivied
P_SexisMale_notSurvived= np.sum((Sex_train[Survived_train==0]=="male"))/np.sum((Survived_train==0))
P_SexisFemale_notSurvived=np.sum((Sex_train[Survived_train==0]=="female"))/np.sum((Survived_train==0))
##Tính xác xuất hậu nghiệm trên tập (P(X|notSurvived) trên tập test)
P_Sex_notSurvived= P_SexisMale_notSurvived*(Sex_test=="male").astype(float) +P_SexisFemale_notSurvived*(Sex_test=="female").astype(float)

In [10]:
#Sex
##Survived
P_pc1_Survived= np.sum((Pclass_train[Survived_train==1]==1))/np.sum((Survived_train==1))
P_pc2_Survived=np.sum((Pclass_train[Survived_train==1]==2))/np.sum((Survived_train==1))
P_pc3_Survived=np.sum((Pclass_train[Survived_train==1]==3))/np.sum((Survived_train==1))

##Tính xác xuất hậu nghiệm trên tập (P(X|Survived) trên tập test)
P_pc_Survived= P_pc1_Survived*(Pclass_test==1).astype(float) +P_pc2_Survived*(Pclass_test==2).astype(float) +P_pc3_Survived*(Pclass_test==3).astype(float)
#P_pc_Survived= P_pc1_Survived*(Pclass_test=="1").astype(float) +P_pc2_Survived*(Pclass_test=="2").astype(float) +P_pc3_Survived*(Pclass_test=="3").astype(float)

##notSurvivied
P_pc1_notSurvived= np.sum((Pclass_train[Survived_train==0]==1))/np.sum((Survived_train==0))
P_pc2_notSurvived=np.sum((Pclass_train[Survived_train==0]==2))/np.sum((Survived_train==0))
P_pc3_notSurvived=np.sum((Pclass_train[Survived_train==0]==3))/np.sum((Survived_train==0))
##Tính xác xuất hậu nghiệm trên tập (P(X|notSurvived) trên tập test)
P_pc_notSurvived= P_pc1_notSurvived*(Pclass_test==1).astype(float) +P_pc2_notSurvived*(Pclass_test==2).astype(float) +P_pc3_notSurvived*(Pclass_test==3).astype(float)
#P_pc_notSurvived= P_pc1_notSurvived*(Pclass_test=="1").astype(float) +P_pc2_notSurvived*(Pclass_test=="2").astype(float) +P_pc3_notSurvived*(Pclass_test=="3").astype(float)

In [11]:
##Survived
P_EmbarkedS_Survived= np.sum((Embarked_train[Survived_train==1]=='S'))/np.sum((Survived_train==1))
P_EmbarkedC_Survived=np.sum((Embarked_train[Survived_train==1]=='C'))/np.sum((Survived_train==1))
P_EmbarkedQ_Survived=np.sum((Embarked_train[Survived_train==1]=='Q'))/np.sum((Survived_train==1))

##Tính xác xuất hậu nghiệm trên tập (P(X|Survived) trên tập test)
P_Embarked_Survived= P_EmbarkedS_Survived*(Embarked_test=='S').astype(float) +P_EmbarkedC_Survived*(Embarked_test=='C').astype(float) +P_EmbarkedQ_Survived*(Embarked_test=='Q').astype(float)
#P_pc_Survived= P_pc1_Survived*(Pclass_test=="1").astype(float) +P_pc2_Survived*(Pclass_test=="2").astype(float) +P_pc3_Survived*(Pclass_test=="3").astype(float)

##notSurvivied
P_EmbarkedS_notSurvived= np.sum((Embarked_train[Survived_train==0]=='S'))/np.sum((Survived_train==0))
P_EmbarkedC_notSurvived=np.sum((Embarked_train[Survived_train==0]=='C'))/np.sum((Survived_train==0))
P_EmbarkedQ_notSurvived=np.sum((Embarked_train[Survived_train==0]=='Q'))/np.sum((Survived_train==0))
##Tính xác xuất hậu nghiệm trên tập (P(X|notSurvived) trên tập test)
P_Embarked_notSurvived= P_EmbarkedS_notSurvived*(Embarked_test=='S').astype(float)+P_EmbarkedC_notSurvived*(Embarked_test=='C').astype(float) +P_EmbarkedQ_notSurvived*(Embarked_test=='Q').astype(float)

In [12]:
P_Survived_combine_1=P_Survived*(P_Sex_Survived*P_pc_Survived*P_Embarked_Survived)
P_notSurvived_combine_1=P_notSurvived*(P_Sex_notSurvived*P_pc_notSurvived*P_Embarked_notSurvived)

y_pred_multi=(P_Survived_combine_1>P_notSurvived_combine_1).astype(int)

# Tính accuracy score
metrics.accuracy_score(Survived_test,y_pred_multi)
#np.mean((P_Survived_combine_1>P_notSurvived_combine_1).astype(float)==Survived_test)

0.7808988764044944

In [13]:

train1=data1_train.drop(['Survived'],axis=1)
test1= data1_test.drop(['Survived'],axis=1)

In [14]:
from sklearn.preprocessing import LabelEncoder
LE=LabelEncoder()
train1['Sex']=LE.fit_transform(train1['Sex'])
train1['Pclass']=LE.fit_transform(train1['Pclass'])
train1['Embarked']=LE.fit_transform(train1['Embarked'])


test1['Sex']=LE.fit_transform(test1['Sex'])
test1['Pclass']=LE.fit_transform(test1['Pclass'])
test1['Embarked']=LE.fit_transform(test1['Embarked'])

In [15]:
from sklearn.naive_bayes import CategoricalNB

nb = CategoricalNB().fit(train1, Survived_train)
nb.score(test1,Survived_test)

0.7808988764044944

## 2. Dùng các features: SibSp,	Parch, và Fare để xây dựng thuật toán Naive Bayes Gaussian (Khỏi normalize). 

### Gaussian
$$p(x_i|c)=P(x_i|\mu_{ci},\sigma_{ci}^2)=\frac{1}{\sqrt{2\pi\sigma_{ci}^2}}exp(-\frac{(x_i-\mu_{ci})^2}{2\sigma_{ci}^2})$$

In [16]:


## tính tham số
Sibsp_mean_Survived = np.mean(SibSp_train[Survived_train==1])
Sibsp_std_Survived = np.std(SibSp_train[Survived_train==1])

Sibsp_mean_notSurvived = np.mean(SibSp_train[Survived_train==0])
Sibsp_std_notSurvived = np.std(SibSp_train[Survived_train==0])

def normal(x, mean, std):
    return 1/(np.sqrt(2*np.pi)*std)*np.exp(-np.square(x-mean)/(2*np.square(std)))

## P(X|Sur) và P(X|notSur)
P_SibSp_Survived = normal(SibSp_test, Sibsp_mean_Survived , Sibsp_std_Survived )
P_SibSp_notSurvived = normal(SibSp_test, Sibsp_mean_notSurvived, Sibsp_std_notSurvived)


#P_Survived_combine=P_Survived*P_SibSp_Survived
#P_notSurvived_combine= P_notSurvived*P_SibSp_notSurvived

# print(np.mean((P_Survived_combine>P_notSurvived_combine).astype(int)==Survived_test))

# ## Dùng sklearn
# from sklearn.naive_bayes import GaussianNB
# nb = GaussianNB().fit(SibSp_train.reshape(-1,1), Survived_train)
# print(np.mean(Survived_test == nb.predict(SibSp_test.reshape(-1,1))))


In [17]:


## tính tham số
Parch_mean_Survived = np.mean(Parch_train[Survived_train==1])
Parch_std_Survived = np.std(Parch_train[Survived_train==1])

Parch_mean_notSurvived = np.mean(Parch_train[Survived_train==0])
Parch_std_notSurvived = np.std(Parch_train[Survived_train==0])


## P(X|Sur) và P(X|notSur)
P_Parch_Survived = normal(Parch_test, Parch_mean_Survived , Parch_std_Survived )
P_Parch_notSurvived = normal(Parch_test, Parch_mean_notSurvived, Parch_std_notSurvived)


# P_Survived_combine=P_Survived*P_Parch_Survived
# P_notSurvived_combine= P_notSurvived*P_Parch_notSurvived

# print(np.mean((P_Survived_combine<P_notSurvived_combine).astype(int)==Survived_test))

# ## Dùng sklearn
# from sklearn.naive_bayes import GaussianNB
# nb = GaussianNB().fit(Parch_train.reshape(-1,1), Survived_train)
# print(np.mean(Survived_test == nb.predict(Parch_test.reshape(-1,1))))

In [18]:


## tính tham số
Fare_mean_Survived = np.mean(Fare_train[Survived_train==1])
Fare_std_Survived = np.std(Fare_train[Survived_train==1])

Fare_mean_notSurvived = np.mean(Fare_train[Survived_train==0])
Fare_std_notSurvived = np.std(Fare_train[Survived_train==0])


## P(X|Sur) và P(X|notSur)
P_Fare_Survived = normal(Fare_test, Fare_mean_Survived , Fare_std_Survived )
P_Fare_notSurvived = normal(Fare_test, Fare_mean_notSurvived, Fare_std_notSurvived)


P_Survived_combine=P_Survived*P_SibSp_Survived*P_Parch_Survived*P_Fare_Survived
P_notSurvived_combine= P_notSurvived*P_SibSp_notSurvived*P_Parch_notSurvived*P_Fare_notSurvived

print(np.mean((P_Survived_combine>P_notSurvived_combine).astype(int)==Survived_test))
# print(((P_Survived_combine>P_notSurvived_combine).astype(int)==Survived_test))

# ## Dùng sklearn
# from sklearn.naive_bayes import GaussianNB
# nb = GaussianNB().fit(Fare_train.reshape(-1,1), Survived_train)
# print(np.mean(Survived_test == nb.predict(Fare_test.reshape(-1,1))))
# #print((Survived_test == nb.predict(Fare_test.reshape(-1,1))))


0.6910112359550562


In [19]:
# Compare with scikit-learn
data2_train=data_train[['SibSp','Parch','Fare','Survived']]
data2_test=data_test[['SibSp','Parch','Fare','Survived']]

train2=data2_train.drop(['Survived'],axis=1)
test2= data2_test.drop(['Survived'],axis=1)
from sklearn.naive_bayes import GaussianNB

nb = CategoricalNB().fit(train2, Survived_train)
nb.score(test2,Survived_test)

0.6797752808988764

## 3. Từ bộ dữ liệu liên minh hãy xây dựng thuật toán Naive Bayes bằng cách dùng tất cả features (mix cả Multinomial và Gaussian. Lưu ý: Khỏi chia tập train, test. Dự đoán thẳng trên tập train)

In [20]:
#Chuẩn bị data cho câu 3
import pandas as pd
df = pd.read_csv("https://raw.githubusercontent.com/dinhvietcuong1996/Lab-MachineLearningCourse/master/Lab04/lienminh.csv")
df.head()

,killsDiff,minionsKilledDiff,wardPlacedDiff,firstBlood,heralds,dragons,teamWins
0,3,-2,13,blue,none,none,red
1,0,-66,0,red,red,red,red
2,-4,-17,0,red,none,blue,red
3,-1,-34,28,red,blue,none,red
4,0,-15,58,red,none,red,red


In [21]:
# Split data by using the sklearn
from sklearn.model_selection import train_test_split
df_train, df_test = train_test_split(df, test_size=0.2, 
                                        random_state=42)

In [22]:
# Analize the value in each feature
for i in df_train.columns:
    print(i)
    print(df_train[i].unique())

killsDiff
[ -2  -5   3  -3   6   4   1   5  -1   2  -7  -4  -8   0  10   8  -6  12
   9  -9   7 -10 -13  11 -12  16 -11 -15  13  14 -17  15 -14]
minionsKilledDiff
[ -61  -30  -36    7   16   -2   15    2  -19   25  -45    6  -43    9
   27   -3  -62   30  -17    5  -82  -18   -9  -27   -4  -14   14   57
   37  -76  -54  -22   23   31  -12  -55   55  -25  -40   51  -37   -8
  -39    8   19   -6  -10   -1   13   10  -24  -51   24   41  -26  -13
   32    3  -50    0   26   -5   12   38   21   68   74  -49  -16  -11
   44  -23  -48   40   43  -29   18    4   17   48  -32  -21   61   22
   46  -31   28  -52   92  -20  -15    1  -60   35  -47  -68  -41   11
   20   -7  -35   34  -59  -42   58  -44   49   63   29   60  -38  -34
   47   33   54   42   45  -57   62  -66   36   52  -33   39  -28  -64
  -70  -58   85   50   69   59  -81  -46   86  -90   65  -56  -53   67
   77  104  -69  -83  -78  -63   70  -74  -65  -77   98  -72  -86 -113
   78  -73   87   89  -79  117   53   82   72   80 -111 

In [23]:
killsDiff = df['killsDiff']
minionsKilledDiff = df['minionsKilledDiff']
wardPlacedDiff	 = df['wardPlacedDiff']
firstBlood = df['firstBlood']
heralds = df['heralds']
dragons = df['dragons']

teamWins = df['teamWins']

In [24]:
# killsDiff_train, minionsKilledDiff_train, wardPlacedDif_train, firstBlood_train, heralds_train,dragons_train,teamWins_train=LOL_feature(data = df_train)
# killsDiff_test, minionsKilledDiff_test, wardPlacedDif_test, firstBlood_test, heralds_test,dragons_test,teamWins_test=LOL_feature(data = df_test)

In [25]:
# Tính xác suất từ label
P_blueWins=teamWins.value_counts()['blue']/len(teamWins)
P_redWins=teamWins.value_counts()['red']/len(teamWins)

#### Gaussian

In [26]:


## killsDiff
killsDiff_mean_blueWins=np.mean(killsDiff[teamWins=='blue'])
killsDiff_std_blueWins=np.std(killsDiff[teamWins=='blue'])

killsDiff_mean_redWins=np.mean(killsDiff[teamWins=='red'])
killsDiff_std_redWins=np.std(killsDiff[teamWins=='red'])

## minionsKilledDiff
minionsKilledDiff_mean_blueWins=np.mean(minionsKilledDiff[teamWins=='blue'])
minionsKilledDiff_std_blueWins=np.std(minionsKilledDiff[teamWins=='blue'])

minionsKilledDiff_mean_redWins=np.mean(minionsKilledDiff[teamWins=='red'])
minionsKilledDiff_std_redWins=np.std(minionsKilledDiff[teamWins=='red'])

## wardPlacedDiff
wardPlacedDiff_mean_blueWins=np.mean(wardPlacedDiff[teamWins=='blue'])
wardPlacedDiff_std_blueWins=np.std(wardPlacedDiff[teamWins=='blue'])

wardPlacedDiff_mean_redWins=np.mean(wardPlacedDiff[teamWins=='red'])
wardPlacedDiff_std_redWins=np.std(wardPlacedDiff[teamWins=='red'])




In [27]:
### P(X|blue) và P(X|red)

## killsDiff
P_killsDiff_blueWins=normal(killsDiff,killsDiff_mean_blueWins,killsDiff_std_blueWins)
P_killsDiff_redWins=normal(killsDiff,killsDiff_mean_redWins,killsDiff_std_redWins)

## minionsKilledDiff
P_minionsKilledDiff_blueWins=normal(minionsKilledDiff,minionsKilledDiff_mean_blueWins,minionsKilledDiff_std_blueWins)
P_minionsKilledDiff_redWins=normal(minionsKilledDiff,minionsKilledDiff_mean_redWins,minionsKilledDiff_std_redWins)

## wardPlacedDiff
P_wardPlacedDiff_blueWins=normal(wardPlacedDiff,wardPlacedDiff_mean_blueWins,wardPlacedDiff_std_blueWins)
P_wardPlacedDiff_redWins=normal(wardPlacedDiff,wardPlacedDiff_mean_redWins,wardPlacedDiff_std_redWins)

### Multinomial

In [28]:
####  Multinomial
### firstBlood

## blueWins
P_fB_bl_blueWins= np.sum((firstBlood[teamWins=='blue']=="blue"))/np.sum((teamWins=='blue'))
P_fB_r_blueWins=np.sum((firstBlood[teamWins=='blue']=="red"))/np.sum((teamWins=='blue'))

## P(X|blue)
P_fB_blueWins= P_fB_bl_blueWins*(firstBlood=="blue").astype(float) +P_fB_r_blueWins*(firstBlood=="red").astype(float)


## redWins
P_fB_bl_redWins= np.sum((firstBlood[teamWins=='red']=="blue"))/np.sum((teamWins=='red'))
P_fB_r_redWins=np.sum((firstBlood[teamWins=='red']=="red"))/np.sum((teamWins=='red'))

## P(X|blue)
P_fB_redWins= P_fB_bl_redWins*(firstBlood=="blue").astype(float) +P_fB_r_redWins*(firstBlood=="red").astype(float)


### heralds

## blueWins
P_h_none_blueWins= np.sum((heralds[teamWins=='blue']=="none"))/np.sum((teamWins=='blue'))
P_h_bl_blueWins= np.sum((heralds[teamWins=='blue']=="blue"))/np.sum((teamWins=='blue'))
P_h_r_blueWins=np.sum((heralds[teamWins=='blue']=="red"))/np.sum((teamWins=='blue'))

## P(X|blue)
P_h_blueWins= P_h_none_blueWins*(heralds=="none").astype(float)+P_h_bl_blueWins*(heralds=="blue").astype(float) +P_h_r_blueWins*(heralds=="red").astype(float)


## redWins
P_h_none_redWins= np.sum((heralds[teamWins=='red']=="none"))/np.sum((teamWins=='red'))
P_h_bl_redWins= np.sum((heralds[teamWins=='red']=="blue"))/np.sum((teamWins=='red'))
P_h_r_redWins=np.sum((heralds[teamWins=='red']=="red"))/np.sum((teamWins=='red'))

## P(X|blue)
P_h_redWins= P_h_none_redWins*(heralds=="none").astype(float)+P_h_bl_redWins*(heralds=="blue").astype(float) +P_h_r_redWins*(heralds=="red").astype(float)


### dragons

## blueWins
P_dr_none_blueWins= np.sum((dragons[teamWins=='blue']=="none"))/np.sum((teamWins=='blue'))
P_dr_bl_blueWins= np.sum((dragons[teamWins=='blue']=="blue"))/np.sum((teamWins=='blue'))
P_dr_r_blueWins=np.sum((dragons[teamWins=='blue']=="red"))/np.sum((teamWins=='blue'))

## P(X|blue)
P_dr_blueWins= P_dr_none_blueWins*(dragons=="none").astype(float)+P_dr_bl_blueWins*(dragons=="blue").astype(float) +P_dr_r_blueWins*(dragons=="red").astype(float)


## redWins
P_dr_none_redWins= np.sum((dragons[teamWins=='red']=="none"))/np.sum((teamWins=='red'))
P_dr_bl_redWins= np.sum((dragons[teamWins=='red']=="blue"))/np.sum((teamWins=='red'))
P_dr_r_redWins=np.sum((dragons[teamWins=='red']=="red"))/np.sum((teamWins=='red'))

## P(X|blue)
P_dr_redWins= P_dr_none_redWins*(dragons=="none").astype(float)+P_dr_bl_redWins*(dragons=="blue").astype(float) +P_dr_r_redWins*(dragons=="red").astype(float)

In [29]:
P_blueWins_combine=P_blueWins*P_killsDiff_blueWins*P_minionsKilledDiff_blueWins*P_wardPlacedDiff_blueWins*P_fB_blueWins*P_h_blueWins*P_dr_blueWins
P_redWins_combine=P_redWins*P_killsDiff_redWins*P_minionsKilledDiff_redWins*P_wardPlacedDiff_redWins*P_fB_redWins*P_h_redWins*P_dr_redWins

# Kết quả predict
y_pred=(P_blueWins_combine>P_redWins_combine).astype(int)

temp=teamWins.copy()
for i in range(len(df)):
    if(temp[i]=='blue'):
        temp[i]=1
    else:
        temp[i]=0
y_test=pd.Series(temp,dtype=int)

# Calculate accuracy score
metrics.accuracy_score(y_test,y_pred)

0.7116104868913857